In [1]:
!pip install -q transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 18.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 18.0.0 which is incompatible.


In [2]:
!huggingface-cli login --token hf_

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer

In [13]:
import torch
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr

def compute_metrics2(predictions, labels):
    mse = mean_squared_error(labels, predictions)
    mae = mean_absolute_error(labels, predictions)
    pearson_corr, _ = pearsonr(predictions, labels)
    spearman_corr, _ = spearmanr(predictions, labels)
    cosine_sim = torch.nn.functional.cosine_similarity(torch.tensor(predictions), torch.tensor(labels), dim=0).mean().item()

    return {
        "mse": mse,
        "mae": mae,
        "pearson_corr": pearson_corr,
        "spearman_corr": spearman_corr,
        "cosine_sim": cosine_sim  # Optional metric for similarity tasks
    }

# minoosh/bert-reg-crossencoder-mse"

In [49]:
model_name = "minoosh/bert-reg-crossencoder-mse"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Initialize the Trainer
trainer = Trainer(model=model)

In [50]:
# Load dataset
dataset = load_dataset("minoosh/Annotated_story_pairs2")

# Preprocess data for the cross-encoder model by concatenating text1 and text2 with [SEP]
def preprocess_function(examples):
    # Concatenate both texts with a [SEP] token in between
    encodings = tokenizer(examples['text1'], examples['text2'], truncation=True, padding=True, max_length=512)
    encodings['labels'] = examples['label']
    return encodings

# Apply tokenization
#tokenized_train = dataset['train'].map(preprocess_function, batched=True)
tokenized_test = dataset['test'].map(preprocess_function, batched=True)
#tokenized_val = dataset['validation'].map(preprocess_function, batched=True)

# Set format for PyTorch
#tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
#tokenized_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [51]:
prediction = trainer.predict(tokenized_test)
prediction

PredictionOutput(predictions=array([[0.49645495],
       [0.70597875],
       [0.33234525],
       [0.5911986 ],
       [0.553577  ],
       [0.6251458 ],
       [0.564947  ],
       [0.5544381 ],
       [0.54482955],
       [0.65693057],
       [0.5690245 ],
       [0.61431044],
       [0.4838075 ],
       [0.3976925 ],
       [0.6415434 ],
       [0.6653182 ],
       [0.6265855 ],
       [0.3111832 ],
       [0.65014213],
       [0.48904845],
       [0.6771684 ],
       [0.54215693],
       [0.5762899 ],
       [0.6841067 ],
       [0.61757445],
       [0.69951427],
       [0.39096075],
       [0.32032046],
       [0.5429218 ],
       [0.6403567 ],
       [0.39980355],
       [0.6589362 ],
       [0.34071764],
       [0.6393186 ],
       [0.39182013],
       [0.7180848 ],
       [0.5009824 ],
       [0.45757926],
       [0.6073381 ],
       [0.25381687],
       [0.29575747],
       [0.6776214 ],
       [0.24283719],
       [0.53304726],
       [0.6562239 ],
       [0.64118844],
     

In [52]:
actual_labels = tokenized_test['labels'].tolist()
predicted_classes = prediction.predictions.squeeze().tolist()
res = compute_metrics2(actual_labels, predicted_classes)

res

{'mse': 0.10376882405505974,
 'mae': 0.2747747827321291,
 'pearson_corr': 0.21029515120828404,
 'spearman_corr': 0.1443849884052918,
 'cosine_sim': 0.877231776714325}

# "minoosh/bert-reg-crossencoder-mae"

In [65]:
model_name = "minoosh/bert-reg-crossencoder-mae"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Initialize the Trainer
trainer = Trainer(model=model)

In [69]:
# Load dataset
dataset = load_dataset("minoosh/Annotated_story_pairs2")

# Preprocess data for the cross-encoder model by concatenating text1 and text2 with [SEP]
def preprocess_function(examples):
    # Concatenate both texts with a [SEP] token in between
    encodings = tokenizer(examples['text1'], examples['text2'], truncation=True, padding=True, max_length=512)
    encodings['labels'] = examples['label']
    return encodings

# Apply tokenization
#tokenized_train = dataset['train'].map(preprocess_function, batched=True)
tokenized_test = dataset['test'].map(preprocess_function, batched=True)
#tokenized_val = dataset['validation'].map(preprocess_function, batched=True)

# Set format for PyTorch
#tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
#tokenized_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [70]:
prediction = trainer.predict(tokenized_test)
prediction

PredictionOutput(predictions=array([[0.50453377],
       [0.6263789 ],
       [0.27635142],
       [0.5034347 ],
       [0.5760099 ],
       [0.6512682 ],
       [0.45011228],
       [0.4388314 ],
       [0.5487585 ],
       [0.55954033],
       [0.6058875 ],
       [0.59040457],
       [0.5171    ],
       [0.5779132 ],
       [0.4469051 ],
       [0.68225133],
       [0.64959705],
       [0.4905397 ],
       [0.57506615],
       [0.5975526 ],
       [0.58912295],
       [0.60666245],
       [0.4432458 ],
       [0.66442806],
       [0.48260155],
       [0.57679677],
       [0.4678212 ],
       [0.43483946],
       [0.6639946 ],
       [0.6597029 ],
       [0.4745272 ],
       [0.6437773 ],
       [0.5416583 ],
       [0.6405684 ],
       [0.5642117 ],
       [0.6865468 ],
       [0.5156758 ],
       [0.59000057],
       [0.47591105],
       [0.3279172 ],
       [0.36487362],
       [0.6385168 ],
       [0.42913023],
       [0.58419794],
       [0.56218463],
       [0.56327796],
     

In [71]:
actual_labels = tokenized_test['labels'].tolist()
predicted_classes = prediction.predictions.squeeze().tolist()
res = compute_metrics2(actual_labels, predicted_classes)

res

{'mse': 0.09339308267594623,
 'mae': 0.26419434919953344,
 'pearson_corr': 0.27998061048451983,
 'spearman_corr': 0.2651050424875487,
 'cosine_sim': 0.8915033936500549}

# "minoosh/bert-reg-crossencoder-contrastive"

In [72]:
model_name = "minoosh/bert-reg-crossencoder-contrastive"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Initialize the Trainer
trainer = Trainer(model=model)

In [73]:
# Load dataset
dataset = load_dataset("minoosh/Annotated_story_pairs2")

# Preprocess data for the cross-encoder model by concatenating text1 and text2 with [SEP]
def preprocess_function(examples):
    # Concatenate both texts with a [SEP] token in between
    encodings = tokenizer(examples['text1'], examples['text2'], truncation=True, padding=True, max_length=512)
    encodings['labels'] = examples['label']
    return encodings

# Apply tokenization
#tokenized_train = dataset['train'].map(preprocess_function, batched=True)
tokenized_test = dataset['test'].map(preprocess_function, batched=True)
#tokenized_val = dataset['validation'].map(preprocess_function, batched=True)

# Set format for PyTorch
#tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
#tokenized_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [74]:
prediction = trainer.predict(tokenized_test)
prediction

PredictionOutput(predictions=array([[1.009588  ],
       [1.0049514 ],
       [1.0121336 ],
       [0.9995622 ],
       [0.9654851 ],
       [1.0107651 ],
       [0.9951133 ],
       [1.0082967 ],
       [1.0008017 ],
       [0.979575  ],
       [0.9848868 ],
       [0.98630303],
       [0.9860636 ],
       [0.97186244],
       [0.9939249 ],
       [0.9815468 ],
       [0.9894565 ],
       [0.9853045 ],
       [1.0046353 ],
       [0.9939396 ],
       [0.99100155],
       [0.9841576 ],
       [0.98468906],
       [1.0027732 ],
       [0.9834165 ],
       [0.9878249 ],
       [0.98290133],
       [0.97674185],
       [0.98101187],
       [1.0153706 ],
       [0.9995818 ],
       [0.9881976 ],
       [0.9873743 ],
       [0.990062  ],
       [1.0156038 ],
       [1.0207397 ],
       [0.9865587 ],
       [1.0011867 ],
       [0.97585636],
       [1.0245094 ],
       [0.9974881 ],
       [0.9895072 ],
       [1.0031236 ],
       [0.98287183],
       [1.004801  ],
       [0.98281586],
     

In [60]:
actual_labels = tokenized_test['labels'].tolist()
predicted_classes = prediction.predictions.squeeze().tolist()
res = compute_metrics2(actual_labels, predicted_classes)

res

{'mse': 0.2631469950741542,
 'mae': 0.40945145264267924,
 'pearson_corr': 0.02983823910506629,
 'spearman_corr': 0.007270407086018862,
 'cosine_sim': 0.8820459842681885}